# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [2]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [4]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,la passe,45.5549,-0.8967,6.14,76,66,3.38,FR,1739573700
1,1,al jawf,29.5000,38.7500,8.59,48,30,4.11,SA,1739573701
2,2,sambhar,26.9167,75.2000,16.39,37,51,1.52,IN,1739573702
3,3,port-aux-francais,-49.3500,70.2167,7.26,96,99,14.12,TF,1739573703
4,4,sao joao da barra,-21.6403,-41.0511,26.05,89,45,4.09,BR,1739573704


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [ ]:
%%capture --no-display
# Configure the map plot
map_plot = city_data_df.hvplot.points(
    'Lng', 
    'Lat', 
    geo=True, 
    size='Humidity',  # Size of the points based on humidity
    scale=0.8,        # Adjust the scale of the points
    color='City',     # Color the points by city
    alpha=0.5,        # Set transparency of the points
    tiles='OSM',      # Use OpenStreetMap tiles
    frame_width=700,
    frame_height=500,
    title='City Humidity Map'
)

# Display the map
map_plot

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [ ]:
# Narrow down cities that fit criteria
# Example criteria: Temperature between 20°F and 30°F, Humidity < 100%, Wind Speed < 4 mph, Cloudiness < 0%
ideal_weather_df = city_data_df.loc[
    (city_data_df['Max Temp'] > 21)  & 
    (city_data_df['Max Temp'] < 27) & 
    (city_data_df['Wind Speed'] < 4.5) & 
    (city_data_df['Cloudiness'] == 0 )
]
# Drop any rows with null values
ideal_weather_df = ideal_weather_df.dropna()

# Display sample data
ideal_weather_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
46,46,wailua homesteads,22.0669,-159.3780,24.51,81,0,2.06,US,1739222041
91,91,lihue,21.9789,-159.3672,24.85,78,0,2.06,US,1739222093
202,202,port lincoln,-34.7333,135.8667,22.50,68,0,3.97,AU,1739222433
340,340,saint-pierre,-21.3393,55.4781,25.82,83,0,1.54,RE,1739222637
399,399,kapa'a,22.0752,-159.3190,25.18,82,0,2.06,US,1739222699
401,401,valparaiso,-33.0393,-71.6273,22.80,59,0,4.12,CL,1739222702
465,465,goz beida,13.9500,20.5500,23.55,14,0,3.97,TD,1739222908
539,539,broken hill,-31.9500,141.4333,23.35,31,0,2.36,AU,1739222988
542,542,srivardhan,18.0333,73.0167,24.22,62,0,3.48,IN,1739222992


### Step 3: Create a new DataFrame called `hotel_df`.

In [ ]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = city_data_df[["City", "Country", "Lat", "Lng", "Humidity"]].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df.head()

,City,Country,Lat,Lng,Humidity,Hotel Name
0,lubango,AO,-14.9172,13.4925,99,
1,grytviken,GS,-54.2811,-36.5092,79,
2,carbonear,CA,47.7332,-53.2148,71,
3,college,US,64.8569,-147.8028,77,
4,port-aux-francais,TF,-49.3500,70.2167,95,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [ ]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "categories": "accommodation.hotel",  # Filter by hotel category
    "apiKey": geoapify_key,               # Your Geoapify API key
    "limit": 1                           # Limit results to 1 (first hotel)
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lat = row["Lat"]
    lng = row["Lng"]

    # Add the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{lng},{lat},{radius}"  # Filter by circle around the coordinates
    params["bias"] = f"proximity:{lng},{lat}"  

    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make and API request using the params dictionary
    response = requests.get(base_url, params=params)

    # Convert the API response to JSON format
    response = response.json()

    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = response["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"

    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
lubango - nearest hotel: Novo Hotel
grytviken - nearest hotel: No hotel found
carbonear - nearest hotel: No hotel found
college - nearest hotel: BEST WESTERN PLUS Pioneer Park Inn
port-aux-francais - nearest hotel: Keravel
douglas - nearest hotel: Holiday Inn Express
waitangi - nearest hotel: Hotel Chathams
remire-montjoly - nearest hotel: Complexe Belova
blackmans bay - nearest hotel: Villa Howden
byron bay - nearest hotel: Lord Byron Resort
cabo san lucas - nearest hotel: Comfort Rooms
porto novo - nearest hotel: Hôtel 6500 CFA
aykhal - nearest hotel: Уют
fisterra - nearest hotel: Hotel Temático do Banco Azul
badhni kalan - nearest hotel: Paradise Palace
adamstown - nearest hotel: No hotel found
anapa - nearest hotel: Анапа Лазурная
les anglais - nearest hotel: No hotel found
vadso - nearest hotel: Scandic Vadsø
edinburgh of the seven seas - nearest hotel: No hotel found
olonkinbyen - nearest hotel: No hotel found
belmonte - nearest hotel: No hotel found
port al

,City,Country,Lat,Lng,Humidity,Hotel Name
0,lubango,AO,-14.9172,13.4925,99,Novo Hotel
1,grytviken,GS,-54.2811,-36.5092,79,No hotel found
2,carbonear,CA,47.7332,-53.2148,71,No hotel found
3,college,US,64.8569,-147.8028,77,BEST WESTERN PLUS Pioneer Park Inn
4,port-aux-francais,TF,-49.3500,70.2167,95,Keravel
...,...,...,...,...,...,...
602,baishishan,CN,43.5833,127.5667,98,No hotel found
603,xinyuan,CN,37.3000,99.0000,40,No hotel found
604,swan hill,AU,-35.3378,143.5544,51,Burke and Wills Motor Inn
605,taybad,IR,34.7400,60.7756,81,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [ ]:
%%capture --no-display

# Configure the map plot
map_plot = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    size = "Humidity",
    scale = 1,
    color = "City",
    alpha = 0.5,
    tiles = "OSM",
    frame_width = 700,
    frame_height = 500,
    hover_cols = ["Hotel Name", "Country"]
)

# Display the map
map_plot

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)